In [1]:
# !pip install mne
# !pip install h5py

# !pip install datalad-installer
# !datalad-installer git-annex -m datalad/packages
# !pip install datalad

In [2]:
import sys
import numpy as np
import pandas as pd
import mne
import os
import scipy.io
import scipy.signal 
from functools import partial
import matplotlib.pyplot as plt
import h5py
import pickle
import time

D:\tmp\ipykernel_7984\3889199825.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
channelsList = ['FP1', 'FP2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7', 'F8', 'CP3', 'CP4', 'CP5', 'CP6', 'FZ', 'CZ', 'PZ']
sampleFrequency = 500

In [4]:
class SignalExtraction():

    @staticmethod
    def fn_signal_extraction_openneuro(raw_signal_obj, channels_subset=None):
        dict_x, dict_y = {}, {}
        for channel in raw_signal_obj.ch_names:
            signal_channel = raw_signal_obj[channel]
            y, x = signal_channel
            y, x = y.flatten(), x.flatten()
            dict_y[channel] = y
            dict_x[channel] = x
        signal_df = pd.DataFrame(dict_x)
        signal_df = signal_df[channels_subset] if channels_subset else signal_df
        signal_np = np.array(signal_df).transpose()
        return signal_np
    
    @staticmethod
    def fn_signal_extraction_predict_d006(raw_signal_obj, channels_subset=None):
        raw_signal = raw_signal_obj['EEG'][0][0]
        channels = [i[0][0] for i in raw_signal[21].reshape(-1)]
        signals = raw_signal[15]
        control_treatment = raw_signal[0][0]
        label = raw_signal[-2][0]
        # label_binary = 0 if label == 'no' else (1 if label == 'yes' else -1)
        assert signals.shape[0] == len(channels)
        signals = signals.transpose()
        signal_df = pd.DataFrame(signals, columns=channels)
        signal_df = signal_df[channels_subset] if channels_subset else signal_df
        signal_np = np.array(signal_df).transpose()
        return signal_np, control_treatment, label
    
    @staticmethod
    def read_mat_iterator(path, num: None):
        lst = []
        i = 0
        for root, _, files in os.walk(path):
            for name in files:
                i = i+1
                mat_file = os.path.join(root, name)
                mat = scipy.io.loadmat(mat_file)
                signal_np, control_treatment, label = SignalExtraction.fn_signal_extraction_predict_d006(mat, channelsList)
                lst.append((signal_np, control_treatment, label))
                if num and i == num:
                    return lst
        return lst
    
    
    
    @staticmethod
    def get_participants_data_openneuro(file):
        df_participants = pd.read_csv(file, delimiter="\t")
        return df_participants
    
    @staticmethod
    def get_electrodes_coordinates(file, channelsList):
        df_coordinates = pd.read_csv(file, delimiter="\t")
        df1 = df_coordinates[df_coordinates.name.map(lambda x: x in channelsList)]
        df2 = df1.set_index('name')
        df3 = df2.reindex(channelsList)
        return df3
    
    @staticmethod
    def fn_label_extraction_openneuro(subject, df):
        BDI_obj = df[df.participant_id == subject].BDI
        return BDI_obj.values[0]
    
    @staticmethod
    def read_mne_iterator(path, num: None):
        lst = []
        i = 0
        df = SignalExtraction.get_participants_data_openneuro(path+'participants.tsv')

        def fn_read_EEG(root, run, channelsList):
            set_file = run['set']
            coordiantes_file = run['electrodes']
            name_split = set_file.split('_')
            subject_id = name_split[0]+'_'+name_split[2]
            bdi = SignalExtraction.fn_label_extraction_openneuro(name_split[0], df)
            mne_file = os.path.join(root, set_file)
            mne_obj = mne.io.read_raw_eeglab(mne_file)
            signal_np = SignalExtraction.fn_signal_extraction_openneuro(mne_obj, channelsList)
            coordiantesFile = os.path.join(root, coordiantes_file)
            df_coordinates = SignalExtraction.get_electrodes_coordinates(coordiantesFile, channelsList)
            return subject_id, signal_np, df_coordinates, bdi

        for root, _, files in os.walk(path):
            if root.endswith('eeg'):
                run1, run2 = {}, {}
                for name in files:
                    if '_run-01_' in name:
                        if '_eeg.set' in name:
                            run1['set'] = name
                        if '_electrodes.tsv' in name:
                            run1['electrodes'] = name
                    if '_run-02_' in name:
                        if '_eeg.set' in name:
                            run2['set'] = name
                        if '_electrodes.tsv' in name:
                            run2['electrodes'] = name
                
                print('<<<<<<<Processing {}>>>>>>>'.format(root))
                for run in [run1, run2]:
                    if run:
                        try:
                            subject_id, signal_np, df_coordinates, bdi = fn_read_EEG(root, run, channelsList)
                            lst.append((subject_id, signal_np, df_coordinates, bdi))
                        except Exception:
                            print('<<<<<<<Error in {}>>>>>>>'.format(run))
                            continue
                i = i+1
                if num and i == num:
                    return lst
        return lst

In [5]:
class SignalProcessing():

    @staticmethod
    def bandpass_filter(edges: list[float], sample_rate: float, poles: int, data: np.ndarray):
        sos = scipy.signal.butter(poles, edges, 'bandpass', fs=sample_rate, output='sos')
        filtered_data = scipy.signal.sosfiltfilt(sos, data)
        return filtered_data
    
    @staticmethod
    def notch_filter(frequency_toRemove: float, quality_factor: float, sample_frequency: float, data: np.ndarray):
        b, a = scipy.signal.iirnotch(frequency_toRemove, quality_factor, sample_frequency)
        return b, a
    
    @staticmethod
    def resample(data: np.ndarray, sample_frequency: float, downsample_frequency: float):
        n = data.shape[1]
        num = (n//sample_frequency)*downsample_frequency
        return scipy.signal.resample(data, num, axis=1)
        
    @staticmethod
    def data_augmentation(signal, sample_freq: int, window_size: int, stride_size: int):
        N = signal.shape[1]
        f = window_size*sample_freq
        s = stride_size*sample_freq
        n = (N-f+s)//s
        lst = [signal[:, i*s:i*s+f] for i in range(n)]
        tensor = np.stack(lst, axis=0)
        return tensor
    
    @staticmethod
    def fn_tensor(signal, n_slices):
        signal_slices = np.hsplit(signal, n_slices)
        tensor = np.stack(signal_slices, axis=0)
        return tensor
    
    @staticmethod
    def tensor_generator(signal, sample_freq, time_slice_size):
        n = signal.shape[1]
        assert n%sample_freq == 0
        t = n//sample_freq
        assert t%time_slice_size == 0
        n_time_slices = t//time_slice_size
        return SignalProcessing.fn_tensor(signal, n_time_slices)
    
    @staticmethod
    def signal_processing(np_array, sample_freq, band_freq_range, downsample_freq, slice_size, sub_slice_size):
    
        bandpass_filter = partial(SignalProcessing.bandpass_filter, band_freq_range, sample_freq, 5)
        signal_processed1 = np.apply_along_axis(bandpass_filter, 1, np_array)
        signal_processed2 = SignalProcessing.resample(signal_processed1, sample_freq, downsample_freq)

        window_size = slice_size
        stride_size = window_size//3
        signal_augmented = SignalProcessing.data_augmentation(signal_processed2, downsample_freq, window_size, stride_size)

        signal_tensors = [SignalProcessing.tensor_generator(signal_slice, downsample_freq, sub_slice_size) for signal_slice in signal_augmented]
        tensor = np.stack(signal_tensors, axis=0)

        return tensor
    
    @staticmethod
    def AdjMatrix(df_electrodes_coordinates):
        np_coordinates = np.array(df_electrodes_coordinates)
        _, b = np_coordinates.shape
        lst = []
        for i in range(b):
            f = np_coordinates[:,i]
            d = f.reshape(-1,1)-f.reshape(1,-1)
            d_square = d**(2)
            lst.append(d_square)
        tensor = np.stack(lst, axis=0)
        tensor_dist_sq = tensor.sum(axis=0)
        tensor_dist = tensor_dist_sq**(1/2)
        max, min = tensor_dist.min(), tensor_dist.max()
        tensor_dist_norm = (tensor_dist-min)/(max-min)
        tensor_dist_norm = tensor_dist_norm if np.linalg.det(tensor_dist_norm) else np.array([])
        return tensor_dist_norm
    
    @staticmethod
    def to_bdi_label(bdi):
        bdi_label = None
        if bdi < 7:
            bdi_label = 0
        if bdi >=17:
            bdi_label = 1
        if bdi >= 7 and bdi < 17:
            bdi_label = 1/2
        return bdi_label
    
    @staticmethod
    def to_h5(file_path, subject, signal_array, label, frequency):
        with h5py.File(file_path, "w") as hf:
                hf.create_dataset("subject", data=subject)
                hf.create_dataset("resampled_signal", data=signal_array)
                hf.create_dataset("resample_freq", data=frequency)
                hf.create_dataset("label", data=label)
        return
    
    @staticmethod
    def signal_processing_iterator(data_lst, sample_freq, band_freq_range, downsample_freq, slice_size, sub_slice_size):
        correct_data_lst = []
        error_data_lst = []
        for subject_id, signal_np, df_coordinates, bdi in data_lst:
            signal_processed = SignalProcessing.signal_processing(signal_np, sample_freq, band_freq_range, downsample_freq, slice_size, sub_slice_size)
            Adj_dist_matrix = SignalProcessing.AdjMatrix(df_coordinates)
            bdi_label = SignalProcessing.to_bdi_label(bdi)
            if bdi_label in [0, 1] and Adj_dist_matrix.size:
                correct_data_lst.append((subject_id, signal_processed, Adj_dist_matrix, bdi, bdi_label))
            else:
                error_data_lst.append((subject_id, signal_processed, Adj_dist_matrix, bdi, bdi_label))
            
        return correct_data_lst, error_data_lst

In [6]:
class TrainingData():

    @staticmethod
    def signal_to_trainingData(signal_tuple):
        ls = []
        subject_id, signal_processed, Adj_dist_matrix, bdi, bdi_label = signal_tuple
        for signal_slice in signal_processed:
            ls.append((signal_slice, Adj_dist_matrix, bdi_label))
        return ls
    
    def trainingData_iterator(signal_tuples):
        ls = []
        for signal_tuple in signal_tuples:
            ls = ls+TrainingData.signal_to_trainingData(signal_tuple)
        return ls

In [7]:
root_path =  'D:/Workspace/DepressionDetection_FreeThesis/Local_Development_Branch/'
file_path_openneuro = 'openneuro_Data/'

In [8]:
file_path = root_path+file_path_openneuro

start = time.time()
ls = SignalExtraction.read_mne_iterator(file_path, None)
end = time.time()

print(end - start)

<<<<<<<Processing D:/Workspace/DepressionDetection_FreeThesis/Local_Development_Branch/openneuro_Data/sub-001\eeg>>>>>>>
Reading D:\Workspace\DepressionDetection_FreeThesis\Local_Development_Branch\openneuro_Data\sub-001\eeg\sub-001_task-Rest_run-01_eeg.fdt
Reading D:\Workspace\DepressionDetection_FreeThesis\Local_Development_Branch\openneuro_Data\sub-001\eeg\sub-001_task-Rest_run-02_eeg.fdt
<<<<<<<Processing D:/Workspace/DepressionDetection_FreeThesis/Local_Development_Branch/openneuro_Data/sub-002\eeg>>>>>>>
Reading D:\Workspace\DepressionDetection_FreeThesis\Local_Development_Branch\openneuro_Data\sub-002\eeg\sub-002_task-Rest_run-01_eeg.fdt
Reading D:\Workspace\DepressionDetection_FreeThesis\Local_Development_Branch\openneuro_Data\sub-002\eeg\sub-002_task-Rest_run-02_eeg.fdt


D:\tmp\ipykernel_7984\289501458.py:80: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  mne_obj = mne.io.read_raw_eeglab(mne_file)


<<<<<<<Processing D:/Workspace/DepressionDetection_FreeThesis/Local_Development_Branch/openneuro_Data/sub-003\eeg>>>>>>>
Reading D:\Workspace\DepressionDetection_FreeThesis\Local_Development_Branch\openneuro_Data\sub-003\eeg\sub-003_task-Rest_run-01_eeg.fdt


D:\tmp\ipykernel_7984\289501458.py:80: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  mne_obj = mne.io.read_raw_eeglab(mne_file)


Reading D:\Workspace\DepressionDetection_FreeThesis\Local_Development_Branch\openneuro_Data\sub-003\eeg\sub-003_task-Rest_run-02_eeg.fdt


...

Reading D:\Workspace\DepressionDetection_FreeThesis\Local_Development_Branch\openneuro_Data\sub-101\eeg\sub-101_task-Rest_run-02_eeg.fdt
<<<<<<<Processing D:/Workspace/DepressionDetection_FreeThesis/Local_Development_Branch/openneuro_Data/sub-102\eeg>>>>>>>
Reading D:\Workspace\DepressionDetection_FreeThesis\Local_Development_Branch\openneuro_Data\sub-102\eeg\sub-102_task-Rest_run-01_eeg.fdt
Reading D:\Workspace\DepressionDetection_FreeThesis\Local_Development_Branch\openneuro_Data\sub-102\eeg\sub-102_task-Rest_run-02_eeg.fdt
<<<<<<<Processing D:/Workspace/DepressionDetection_FreeThesis/Local_Development_Branch/openneuro_Data/sub-103\eeg>>>>>>>
Reading D:\Workspace\DepressionDetection_FreeThesis\Local_Development_Branch\openneuro_Data\sub-103\eeg\sub-103_task-Rest_run-01_eeg.fdt
Reading D:\Workspace\DepressionDetection_FreeThesis\Local_Development_Branch\openneuro_Data\sub-103\eeg\sub-103_task-Rest_run-02_eeg.fdt
<<<<<<<Processing D:/Workspace/DepressionDetection_FreeThesis/Local_Devel

D:\tmp\ipykernel_7984\289501458.py:80: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  mne_obj = mne.io.read_raw_eeglab(mne_file)


Reading D:\Workspace\DepressionDetection_FreeThesis\Local_Development_Branch\openneuro_Data\sub-109\eeg\sub-109_task-Rest_run-02_eeg.fdt
<<<<<<<Processing D:/Workspace/DepressionDetection_FreeThesis/Local_Development_Branch/openneuro_Data/sub-110\eeg>>>>>>>
Reading D:\Workspace\DepressionDetection_FreeThesis\Local_Development_Branch\openneuro_Data\sub-110\eeg\sub-110_task-Rest_run-01_eeg.fdt
Reading D:\Workspace\DepressionDetection_FreeThesis\Local_Development_Branch\openneuro_Data\sub-110\eeg\sub-110_task-Rest_run-02_eeg.fdt
<<<<<<<Processing D:/Workspace/DepressionDetection_FreeThesis/Local_Development_Branch/openneuro_Data/sub-111\eeg>>>>>>>
Reading D:\Workspace\DepressionDetection_FreeThesis\Local_Development_Branch\openneuro_Data\sub-111\eeg\sub-111_task-Rest_run-01_eeg.fdt
Reading D:\Workspace\DepressionDetection_FreeThesis\Local_Development_Branch\openneuro_Data\sub-111\eeg\sub-111_task-Rest_run-02_eeg.fdt
<<<<<<<Processing D:/Workspace/DepressionDetection_FreeThesis/Local_Devel

D:\tmp\ipykernel_7984\289501458.py:80: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  mne_obj = mne.io.read_raw_eeglab(mne_file)


Reading D:\Workspace\DepressionDetection_FreeThesis\Local_Development_Branch\openneuro_Data\sub-122\eeg\sub-122_task-Rest_run-02_eeg.fdt
1476.9756548404694


In [9]:
sliceSize = 6
bandFreqRange = [0.5, 100]
downsampleFreq = 200
subSliceSize = 2

start = time.time()
training_data, error_data = SignalProcessing.signal_processing_iterator(ls, sampleFrequency, bandFreqRange, downsampleFreq, sliceSize, subSliceSize)
end = time.time()

print(end - start)

len(training_data)

150.5543818473816


225

In [10]:
X = TrainingData.trainingData_iterator(training_data)

In [11]:
# Distribution of categorical Variable

ls = []
for i in X:
    signal_slice, Adj_dist_matrix, bdi_label = i
    ls.append(bdi_label)

len(ls), sum(ls), len(ls)-sum(ls)

(38223, 13278, 24945)

In [12]:
path = file_path+'TrainingData_Pickle/'+'training_data.p'
pickle.dump(X, open( path, "wb" ))

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

CACHE